# Portfolio Optimization with Scipy
This is an example of how to use **scipy.optimize.minimize** to perform portfolio optimization.  The problem we are trying to solve with Portfolio Optimization is how to determine the optimum asset mix to optimize certai portfolio objectives

*  Minimize Volitility
*  Maximize Returns
*  Maximize the Portfolio's Sharpe Ratio 


## A. Calculate Earnings for Assests in Portfolio

Create list of tickers symbols of assets in the portfolio to optimize.  Also, define the **start** date and **end** date to pull.

In [1]:
import datetime

tickers = sorted(['NVDA','SOXL','OLED','LRCX','TTD','TTWO','BA','SQ'])

start = datetime.datetime(2016, 1, 1)
#start = datetime.datetime.today() - datetime.timedelta(days=365)
end = datetime.datetime.today()

### Download Daily Data
We will use **pandas_datareader** to download the daily **Adjusted Close** price of the assets in the porfolio from Google.

In [2]:
import pandas as pd
import pandas_datareader.data as web

TICKER_DATA = web.DataReader(tickers, 'google', start) #, end)
TICKER_DATA.Close.tail()

,BA,LRCX,NVDA,OLED,SOXL,SQ,TTD,TTWO
Date,,,,,,,,
2017-09-18,253.08,177.31,187.55,136.7,112.71,28.69,60.09,100.82
2017-09-19,252.46,177.12,187.35,142.2,113.88,28.56,61.72,102.38
2017-09-20,255.46,173.02,185.84,140.1,109.08,28.47,59.81,102.96
2017-09-21,256.04,172.28,180.76,136.0,107.75,28.40,59.54,101.72
2017-09-22,256.45,176.73,179.00,132.7,109.29,28.75,59.80,102.84


In [3]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Category20 as plt
output_notebook()

p = figure( title = 'Portfolios Assets' , plot_width=910 , x_axis_type='datetime')
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'
p.toolbar.active_drag = None

for i, a in enumerate(tickers,0):
    p.line(x=pd.DataFrame(TICKER_DATA.Close).index, y=pd.DataFrame(TICKER_DATA.Close)[a], legend=a, color=plt[len(tickers)][i])

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

### Calculate Daily Assets Returns
Determine the percent change from one day to the next, as daily return, and save it to the **RETURNS** data frame.

$$ \hbox{Expected Returns} = \frac{Close - Close_{-1} }{Close_{-1} } $$

In [4]:
RETURNS = TICKER_DATA['Close'].pct_change()[1:]
RETURNS.tail()

,BA,LRCX,NVDA,OLED,SOXL,SQ,TTD,TTWO
Date,,,,,,,,
2017-09-18,0.016386,0.024973,0.041308,0.035214,0.039664,0.006667,0.003507,0.004283
2017-09-19,-0.002450,-0.001072,-0.001066,0.040234,0.010381,-0.004531,0.027126,0.015473
2017-09-20,0.011883,-0.023148,-0.008060,-0.014768,-0.042150,-0.003151,-0.030946,0.005665
2017-09-21,0.002270,-0.004277,-0.027335,-0.029265,-0.012193,-0.002459,-0.004514,-0.012044
2017-09-22,0.001601,0.025830,-0.009737,-0.024265,0.014292,0.012324,0.004367,0.011011


In [5]:
p = figure( title = 'Asset Returns' , plot_width=910 , x_axis_type='datetime')
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Returns'
p.toolbar.active_drag = None

for i, a in enumerate(tickers,0):
    p.line(x=pd.DataFrame(RETURNS).index, y=pd.DataFrame(RETURNS)[a], legend=a, color=plt[len(tickers)][i])

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

### Calculate the average daily returns
For each asset in the portfolio, we need to determine the average returns by calculating the mean of the observed days.

In [6]:
EXPECTED_RETURNS = RETURNS.mean()
EXPECTED_RETURNS

BA      0.002767
LRCX    0.002770
NVDA    0.004600
OLED    0.003791
SOXL    0.004459
SQ      0.003881
TTD     0.003853
TTWO    0.003401
dtype: float64

These returns can be annualized by $$ (( Daily Returns + 1)^{365} - 1) * 100% $$

In [7]:
((EXPECTED_RETURNS + 1)**365 -1)*100

BA      174.131933
LRCX    174.423334
NVDA    433.881817
OLED    297.954269
SOXL    407.381862
SQ      311.238099
TTD     307.053017
TTWO    245.318458
dtype: float64

### Calculate the covariance matrix of the returns
The variance/covariance matrix provides a measure of volatility for the assets (and between assets) in the portfolio.

In [8]:
COVARIANCE = RETURNS.cov()
COVARIANCE

,BA,LRCX,NVDA,OLED,SOXL,SQ,TTD,TTWO
BA,0.000129,0.000043,0.000018,0.000051,0.000096,0.000034,0.000050,0.000016
LRCX,0.000043,0.000259,0.000194,0.000143,0.000431,0.000126,0.000159,0.000099
NVDA,0.000018,0.000194,0.001068,0.000327,0.000770,0.000115,0.000201,0.000192
OLED,0.000051,0.000143,0.000327,0.000980,0.000436,0.000115,0.000237,0.000146
SOXL,0.000096,0.000431,0.000770,0.000436,0.001201,0.000276,0.000382,0.000293
SQ,0.000034,0.000126,0.000115,0.000115,0.000276,0.000465,0.000170,0.000086
TTD,0.000050,0.000159,0.000201,0.000237,0.000382,0.000170,0.001631,0.000100
TTWO,0.000016,0.000099,0.000192,0.000146,0.000293,0.000086,0.000100,0.000346


### Define the Risk Free Rate
We will use the expected daily returns of the **S&P 500**

In [9]:
RISK_FREE_RATE = web.DataReader('IND', 'google', start).pct_change()[1:].Close.mean() # Risk Free Asset Expected returns
RISK_FREE_RATE

0.0022845012717571844

## B. Create a coupe of utility functions


Define a function (**RANDOM_PORTFOLIO_WEIGHTS**) to generate a random mixed portfolio and a function (**PORTFOLIO**) to calculate the portfolio's **mean returns** ($\mu = w * p^T$) and **standard deviation** ($\sigma = \sqrt{w^T * C * w}$).  Where $w$ is a **vector of asset weights** and $C$ is a **covariance matrix**.


In [10]:
import numpy as np

def RANDOM_PORTFOLIO_WEIGHTS(ticker_count):
    ''' 
    Create a portfolio mix without shorts
    '''
    w = np.random.uniform(0,1,ticker_count)
    return w / sum(w)



def PORTFOLIO( weights ):
    ''' 
    Calculates and returns the portfolio 
        Avgerage Daily Returns (mu),  
        Standard deviation of the Daily Returns (sigma), and
        Sharpe Ratio
    '''
    p = np.asmatrix(EXPECTED_RETURNS)
    w = np.asmatrix(weights)
    C = np.asmatrix(COVARIANCE)
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    sharpe = (mu - RISK_FREE_RATE)/sigma
    annualized_returns = ((mu[0,0] + 1)**365 - 1) * 100
    
    return [mu[0,0], sigma[0,0], sharpe[0,0], annualized_returns]
    

Generate a set of **n_portfolios** portfolios, with random mixed parts of the assets in **tickers**, and use the **PORTFOLIO** function to obtain *Expected Daily Returns*, *Standard Deviation* of the Daily Returns, and the portfolio's *Sharpe ratio*.

In [11]:
n_portfolios = 10000
means, stds, sharpe, annu_returns = np.column_stack([
    PORTFOLIO( RANDOM_PORTFOLIO_WEIGHTS(len(tickers)) ) 
    for _ in range(n_portfolios)
])

### Calculate the Efficient Frontier
The Efficient Frontier is the line of maximum returns as a function of volatility for a mix of assets.

http://www.quantandfinancial.com/2013/07/mean-variance-portfolio-optimization.html

In [12]:
from scipy.optimize import minimize

# Given risk-free rate, assets returns and covariances, this function calculates
# min-var frontier and returns its [x,y] points in two arrays
def solve_frontier(R, C):
    def fitness(W, r):
        # For given level of return r, find weights which minimizes
        # portfolio variance.
        mean, var, _, _ = PORTFOLIO(W)
        # Big penalty for not meeting stated portfolio return 
        # effectively serves as optimization constraint
        penalty = 10000 * abs(mean-r)       
        return var + penalty
    frontier_mean, frontier_var, frontier_sharpe, frontier_aret = [], [], [], []
    n = len(R)  # Number of assets in the portfolio
    # Iterate through the range of returns on Y axis
    for r in np.linspace(min(R), max(R), num=50): 
        W = np.ones([n])/n     # start optimization with equal weights
        b_ = [(0,1) for i in range(n)]
        c_ = ({'type':'eq', 'fun': lambda W: sum(W)-1. })
        optimized = minimize(fun=fitness, x0=W, args=r, method='SLSQP', constraints=c_, bounds=b_, options={'ftol': 1e-10})   
        if not optimized.success: 
            raise BaseException(optimized.message)
        # add point to the min-var frontier [x,y] = [optimized.x, r]
        mu, sig, shp, aret = PORTFOLIO(optimized.x)
        frontier_mean.append(mu)
        frontier_var.append(sig)
        frontier_sharpe.append(shp)
        frontier_aret.append(aret)
        
    return frontier_mean, frontier_var, frontier_sharpe, frontier_aret

In [13]:
frontier_mean, frontier_var, frontier_sharpe, frontier_aret = solve_frontier(EXPECTED_RETURNS, COVARIANCE)

### Plot the Markowitz Bullet of Random Mixed Weights Portfolios

Use the expected returns and volatility of the random portfolios to draw the Markowitz Bullet of the porfolios.

In [14]:
p = figure( title = 'Markowitz Bullet for set of Random Portfolios' , plot_width=910 )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns'
p.toolbar.active_drag = None
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='orange')

show(p)

## D. Find the Minimum Volatility Portfolio

The **Scipy minimize** function will need a function with one objective (Variance).  The **getVolatility** function wraps the **PORTFOLIO** function and only returns the expected variance.

In [15]:
def getVolatility(mix):
    return( PORTFOLIO(mix)[1] )

In [16]:
x0 = np.ones(len(tickers)) 


cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})  # MIX SHOULD ADD TO ONE i.e. 'sum(x) = 1' -> 'sum(x) - 1 = 0'


bnds = tuple((0.0, 1.0) for x in x0)  # NO SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1
#bnds = tuple((-1.0, 1.0) for x in x0)  # ALLOW SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1

OPT_VOLATILITY = minimize(getVolatility, x0, method='SLSQP', bounds=bnds, constraints=cons, options={'disp': True, 'ftol': 1e-7})


print( "Lowest Daily Volatility {}".format( np.round(OPT_VOLATILITY.fun, 4) ) )
OPT_VOLATILITY_MIX = pd.DataFrame(np.round( OPT_VOLATILITY.x * 100, 2 ), index=RETURNS.columns, columns=['Percent of Portfolio'])
OPT_VOLATILITY_MIX

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0094782511273276
            Iterations: 26
            Function evaluations: 260
            Gradient evaluations: 26
Lowest Daily Volatility 0.0095


,Percent of Portfolio
BA,60.68
LRCX,13.20
NVDA,0.93
OLED,0.36
SOXL,0.00
SQ,7.84
TTD,0.28
TTWO,16.70


In [17]:
min_volatility_mean , min_volatility_std , min_volatility_sharpe , min_volatility_annualized = PORTFOLIO( OPT_VOLATILITY.x ) 
print( 'Markowitz Bullet for Portfolio with the minimum volatility of {1}, daily returns of {0}, and Sharpe ratio of {2}'.\
           format(np.round(min_volatility_mean,4) , np.round(min_volatility_std,4) , np.round(min_volatility_sharpe,4) ) )

Markowitz Bullet for Portfolio with the minimum volatility of 0.0095, daily returns of 0.003, and Sharpe ratio of 0.0738


In [18]:
p = figure( title = 'Portfolio with the minimum volatility of {1}, annualized returns of {0}%, and Sharpe ratio of {2}'.\
           format(np.round(min_volatility_annualized,2) , np.round(min_volatility_std,4) , np.round(min_volatility_sharpe,4) ) , 
           plot_width=910  )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns' 
 
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='orange')
p.circle(x=min_volatility_std, y=min_volatility_mean, size=14 , color='red' )
p.toolbar.active_drag = None
show(p)

## E. Efficient Portfolio for the highest daily returns

In [19]:
def getReturns(mix):
    return( -PORTFOLIO(mix)[0] )

In [20]:
from scipy.optimize import minimize

x0 = np.ones(len(tickers))  


cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})  # MIX SHOULD ADD TO ONE i.e. 'sum(x) = 1' -> 'sum(x) - 1 = 0'


bnds = tuple((0.0, 1.0) for x in x0)  # NO SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1
#bnds = tuple((-1.0, 1.0) for x in x0)  # ALLOW SHORT... ALL VALUES BETWEEN -1 AND 1

OPT_RETURNS = minimize(getReturns, x0, method='SLSQP', bounds=bnds, constraints=cons, options={'disp': True, 'ftol': 1e-8})


print( "Highest Daily Returns {}".format( np.round(-OPT_RETURNS.fun, 4) ) )
OPT_RETURNS_MIX = pd.DataFrame(np.round( OPT_RETURNS.x * 100, 2 ), index=RETURNS.columns, columns=['Percent of Portfolio'])
OPT_RETURNS_MIX

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.004599598738863646
            Iterations: 27
            Function evaluations: 270
            Gradient evaluations: 27
Highest Daily Returns 0.0046


,Percent of Portfolio
BA,0.0
LRCX,0.0
NVDA,100.0
OLED,0.0
SOXL,0.0
SQ,0.0
TTD,0.0
TTWO,0.0


In [21]:
target_return_mean , target_return_std , target_return_sharpe , target_return_annualized = PORTFOLIO( OPT_RETURNS.x ) 
print( 'Markowitz Bullet for Portfolio with maximum daily returns of {0}, sigma of {1}, and Sharpe ratio {2}'.\
           format(np.round(target_return_mean,4) , np.round(target_return_std,4) , np.round(target_return_sharpe,5) ) )

Markowitz Bullet for Portfolio with maximum daily returns of 0.0046, sigma of 0.0327, and Sharpe ratio 0.07084


In [22]:
p = figure( title = 'Portfolio with maximum annual returns of {0}%, sigma of {1}, and Sharpe ratio {2}'.\
           format(np.round(target_return_annualized,2) , np.round(target_return_std,4) , np.round(target_return_sharpe,4) ) , 
           plot_width=910  )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns'
 
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='orange')
p.circle(x=target_return_std, y=target_return_mean, size=14 , color='red' )
p.toolbar.active_drag = None
show(p)

## F. Optimum Sharpe Ratio

http://www.investopedia.com/terms/s/sharperatio.asp

The **Scipy minimize** function will need a function with one objective (-Sharpe ratio).  The **getSharpe** function wraps the **PORTFOLIO** function and only returns the Sharpe ratio.

In [23]:
def getSharpe(mix):
    return( -PORTFOLIO(mix)[2] )

In [24]:
from scipy.optimize import minimize


x0 = np.ones(len(tickers)) 


cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})  # MIX SHOULD ADD TO ONE i.e. 'sum(x) = 1' -> 'sum(x) - 1 = 0'


bnds = tuple((0.0, 1.0) for x in x0)  # NO SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1
#bnds = tuple((-1.0, 1.0) for x in x0)  # ALLOW SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1

OPT_SHARPE = minimize(getSharpe, x0, method='SLSQP', bounds=bnds, constraints=cons, options={'disp': True, 'ftol': 1e-8})


print( "Highest Sharpe Ratio {}".format( np.round(-OPT_SHARPE.fun, 4) ) )
OPT_SHARPE_MIX = pd.DataFrame(np.round( OPT_SHARPE.x * 100, 2 ), index=RETURNS.columns, columns=['Percent of Portfolio'])
OPT_SHARPE_MIX

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.10514039241803558
            Iterations: 12
            Function evaluations: 121
            Gradient evaluations: 12
Highest Sharpe Ratio 0.1051


,Percent of Portfolio
BA,29.02
LRCX,0.00
NVDA,16.92
OLED,4.14
SOXL,0.00
SQ,28.73
TTD,3.72
TTWO,17.47


In [25]:
best_sharpe_return , best_sharpe_std , best_sharpe , best_sharpe_annualized_return = PORTFOLIO( OPT_SHARPE.x.T ) 
print('Markowitz Bullet for Portfolio with maximum Sharpe ratio of {2}, return of {0}, and sigma of {1}'.\
           format(np.round(best_sharpe_return,4) , np.round(best_sharpe_std,4) , np.round(best_sharpe,4)) )

Markowitz Bullet for Portfolio with maximum Sharpe ratio of 0.1051, return of 0.0036, and sigma of 0.0124


In [26]:
p = figure( title = 'Portfolio with maximum Sharpe ratio of {2}, annualized returns of {0}%, and sigma of {1}'.\
           format(np.round(best_sharpe_annualized_return,2) , np.round(best_sharpe_std,4) , np.round(best_sharpe,4) ) , 
           plot_width=910  )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns'
 
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='orange')
p.circle(x=best_sharpe_std, y=best_sharpe_return, size=14 , color='red' )
p.toolbar.active_drag = None

show(p)